# Electrify_Clusters

### All the necessary Python imports

In [50]:
%matplotlib inline
from pathlib import Path
from openelec import national, util
import numpy as np

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Enter all input data here

In [16]:
folder_input = Path('/home/chris/Documents/Code/openelec_app/openelec_app/static/lesotho')

clusters_file = folder_input / 'clusters.geojson' # must be polygons with attributes pop_sum, area_m2, grid_dist
#clusters_out = folder_input / 'clusters_out1.gpkg'
#network_out = folder_input / 'network_out1.gpkg'

grid_dist_connected = 1000  # clusters within this distance of grid are considered connected

minimum_pop = 100 # exclude any population below this

# off-grid costs
demand_per_person_kwh_month = 6 # 6kWh/month = MTF Tier 2
demand_per_person_kw_peak = demand_per_person_kwh_month / (4*30)  # 130 4hours/day*30days/month based on MTF numbers, should use a real demand curve
mg_gen_cost_per_kw = 4000
mg_cost_per_m2 = 2

# grid costs
cost_wire_per_m = 50
grid_cost_per_m2 = 2

### Read in the clusters file, convert to desired CRS (ostensibly better for distances) and convert to points, filter on population along the way

In [17]:
clusters = national.load_clusters(clusters_file, grid_dist_connected=grid_dist_connected,
                                   minimum_pop=minimum_pop)

In [18]:
clusters.gdp.max()

1694.0

### We then take all the clusters and calculate the optimum network that connects them all together. The ML model returns T_x and T_y containing the start and end points of each new arc created

In [19]:
network, nodes = national.create_network(clusters)

In [9]:
national.model?

### Then we're ready to calculate the optimum grid extension.
This is done by expanding out from each already connected node, finding the optimum connection of nearby nodes. This is then compared to the off-grid cost and if better, these nodes are marked as connected. Then the loop continues until no new connections are found.

In [20]:
network, nodes = national.model(network,
                           nodes,
                           demand_per_person_kw_peak=demand_per_person_kw_peak,
                           mg_gen_cost=mg_gen_cost_per_kw,
                           mg_dist_cost=mg_cost_per_m2,
                           grid_mv_cost=cost_wire_per_m,
                           grid_lv_cost=grid_cost_per_m2)

### And then do a join to get the results back into a polygon shapefile

In [21]:
network_gdf, clusters_joined = national.spatialise(network, nodes, clusters)

In [54]:
import pandas as pd
pd.isna(clusters_joined.loc[1456, 'gdp'])

True

In [55]:
clusters_geojson = util.geojsonify(clusters_joined, property_cols=['type', 'pop', 'area', 'ntl', 'fid', 'grid_dist', 'travel', 'gdp'])

In [56]:
clusters_geojson['features'][1456]

{'geometry': {'coordinates': [[(28.903290682234708, -29.297143157012652),
    (28.90600572238901, -29.29714315701297),
    (28.906273647415237, -29.297153413984653),
    (28.906542601357142, -29.297184086121348),
    (28.90680999415307, -29.29723487804157),
    (28.907073250779046, -29.29730530060096),
    (28.907329836043296, -29.29739467560852),
    (28.907577278997067, -29.297502142350982),
    (28.90781319672719, -29.29762666588758),
    (28.908035317301053, -29.297767047008698),
    (28.908241501643236, -29.29792193379283),
    (28.908429764133523, -29.298089834619358),
    (28.90859829172761, -29.298269132533317),
    (28.908745461416675, -29.298458100819392),
    (28.908869855857777, -29.298654919627793),
    (28.90897027702422, -29.29885769350059),
    (28.909045757744863, -29.299064469626398),
    (28.909095571020764, -29.299273256644405),
    (28.909470420387606, -29.30140337309704),
    (28.909494090235007, -29.301612161924535),
    (28.90949138441825, -29.301818940162324),


In [8]:
#clusters_joined.to_file(str(clusters_out), driver='GPKG')
#network_gdf.to_file(str(network_out), driver='GPKG')

### And display some summary results

In [22]:
new_conns = clusters_joined.loc[clusters_joined['type'] == 'new']
og = clusters_joined.loc[clusters_joined['type'] == 'og']
orig = clusters_joined.loc[clusters_joined['type'] == 'orig']
cost = og['og_cost'].sum() + cost_wire_per_m * network_gdf['len'].sum() + grid_cost_per_m2 * new_conns['area'].sum()

total_modelled_pop = clusters['pop'].sum()
urban_elec_rate = 0.6
currently_electrified = orig['pop'].sum() * urban_elec_rate
new_conn_pop = new_conns['pop'].sum()
off_grid_pop = og['pop'].sum()

print(f'{len(new_conns)} connected')
print(f'{len(og)} off-grid')
print()
print(f'Cost ${cost:,.0f}')
print()
print(f'Modelled pop: {total_modelled_pop:,.0f}')
print(f'Currently electrified: {currently_electrified:,.0f}')
print(f'New connections: {new_conn_pop:,.0f}')
print(f'Off-grid connections {off_grid_pop:,.0f}')

767 connected
736 off-grid

Cost $3,426,290,177

Modelled pop: 2,041,503
Currently electrified: 471,170
New connections: 1,038,907
Off-grid connections 217,314
